In [1]:
import os
import utilities
from maths_objects import (
    Formulas, SiteHierarchies,MarkdownFile, DataSource, FrontMatter, 
    FormulaGroup, FormulaTable)
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies())
formulas_by_year = Formulas(data_source.formulas_by_year())

In [2]:
from IPython.core.display import HTML
from IPython.core.display import Latex

index_formulas = formulas_by_year.filter_by_state_subject_category('NSW', 'Year 9', 'Indices')

tbl = FormulaTable(index_formulas)

HTML((tbl.to_markdown()))


$a^m \times a^n = a^{m+n}$
$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$
$(a^m)^n = a^{m \times n}$
$(ab)^n = a^nb^n$
$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$
$a^0=1$
$a^{-n}=\dfrac{1}{a^n}$
$\left(\dfrac{a}{b}\right)^{-n} = \left(\dfrac{b}{a}\right)^n$
$a^{\frac{m}{n}} =\sqrt[n]{a^m}$
